In [ ]:
import sqlite3
import plyvel
from pathlib import Path
import re
import json
from functions.runHealth import runHealth
from functions.dynamic_analysis import Dynamic

In [ ]:
path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir2000')
con : sqlite3.Connection = sqlite3.connect( str(path.joinpath("crawl-data.sqlite")) )
db = plyvel.DB( str(path.joinpath("leveldb")) )

In [ ]:
n, f = runHealth(con)
print(f"total visits: {n}, failed/incomplete visits: {f}. Success percentage: {round(100* (1 - f/n)) }%")

In [ ]:
for key, value in db:
    print(key)
    print(value, end="\n\n")

In [ ]:
%load_ext sql
symbols = {}
for path in Path("/home/ndanner_plp/OpenWPM/crawl-data").iterdir():
    if path.is_dir() and re.match("^datadir" ,path.name):
        %sql sqlite:///{path}/crawl-data.sqlite
        result = %sql SELECT DISTINCT(symbol) FROM javascript
        if result:
            for symbol in result.DataFrame()['symbol']:
                if symbol in symbols:
                    symbols[symbol].append(path.name)
                else:
                    symbols[symbol] = [path.name]
print( json.dumps( symbols, indent= 4 ) )

In [ ]:
results = Dynamic(con)
print( json.dumps( results, indent= 4 ) )